1368. Minimum Cost to Make at Least One Valid Path in a Grid
Hard
Topics
premium lock icon
Companies
Hint
Given an m x n grid. Each cell of the grid has a sign pointing to the next cell you should visit if you are currently in this cell. The sign of grid[i][j] can be:

1 which means go to the cell to the right. (i.e go from grid[i][j] to grid[i][j + 1])
2 which means go to the cell to the left. (i.e go from grid[i][j] to grid[i][j - 1])
3 which means go to the lower cell. (i.e go from grid[i][j] to grid[i + 1][j])
4 which means go to the upper cell. (i.e go from grid[i][j] to grid[i - 1][j])
Notice that there could be some signs on the cells of the grid that point outside the grid.

You will initially start at the upper left cell (0, 0). A valid path in the grid is a path that starts from the upper left cell (0, 0) and ends at the bottom-right cell (m - 1, n - 1) following the signs on the grid. The valid path does not have to be the shortest.

You can modify the sign on a cell with cost = 1. You can modify the sign on a cell one time only.

Return the minimum cost to make the grid have at least one valid path.

 

Example 1:


Input: grid = [[1,1,1,1],[2,2,2,2],[1,1,1,1],[2,2,2,2]]
Output: 3
Explanation: You will start at point (0, 0).
The path to (3, 3) is as follows. (0, 0) --> (0, 1) --> (0, 2) --> (0, 3) change the arrow to down with cost = 1 --> (1, 3) --> (1, 2) --> (1, 1) --> (1, 0) change the arrow to down with cost = 1 --> (2, 0) --> (2, 1) --> (2, 2) --> (2, 3) change the arrow to down with cost = 1 --> (3, 3)
The total cost = 3.
Example 2:


Input: grid = [[1,1,3],[3,2,2],[1,1,4]]
Output: 0
Explanation: You can follow the path from (0, 0) to (2, 2).
Example 3:


Input: grid = [[1,2],[4,3]]
Output: 1
 

Constraints:

m == grid.length
n == grid[i].length
1 <= m, n <= 100
1 <= grid[i][j] <= 4

In [ ]:
class Solution:
    # Direction vectors: right, left, down, up (matching grid values 1,2,3,4)
    _dirs = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    def minCost(self, grid: List[List[int]]) -> int:
        num_rows, num_cols = len(grid), len(grid[0])

        # Min-heap ordered by cost. Each element is (cost, row, col)
        pq = [(0, 0, 0)]  # Using list as heap, elements are tuples
        min_cost = [[float("inf")] * num_cols for _ in range(num_rows)]
        min_cost[0][0] = 0

        while pq:
            cost, row, col = heapq.heappop(pq)

            # Skip if we've found a better path to this cell
            if min_cost[row][col] != cost:
                continue

            # Try all four directions
            for d, (dx, dy) in enumerate(self._dirs):
                new_row, new_col = row + dx, col + dy

                # Check if new position is valid
                if 0 <= new_row < num_rows and 0 <= new_col < num_cols:
                    # Add cost=1 if we need to change direction
                    new_cost = cost + (d != (grid[row][col] - 1))

                    # Update if we found a better path
                    if min_cost[new_row][new_col] > new_cost:
                        min_cost[new_row][new_col] = new_cost
                        heapq.heappush(pq, (new_cost, new_row, new_col))

        return min_cost[num_rows - 1][num_cols - 1]

In [ ]:
class Solution:
    def minCost(self, grid: List[List[int]]) -> int:
        num_rows, num_cols = len(grid), len(grid[0])

        # Initialize all cells with max value
        min_changes = [[float("inf")] * num_cols for _ in range(num_rows)]
        min_changes[0][0] = 0

        while True:
            # Store previous state to check for convergence
            prev_state = [row[:] for row in min_changes]

            # Forward pass: check cells coming from left and top
            for row in range(num_rows):
                for col in range(num_cols):
                    # Check cell above
                    if row > 0:
                        min_changes[row][col] = min(
                            min_changes[row][col],
                            min_changes[row - 1][col]
                            + (0 if grid[row - 1][col] == 3 else 1),
                        )
                    # Check cell to the left
                    if col > 0:
                        min_changes[row][col] = min(
                            min_changes[row][col],
                            min_changes[row][col - 1]
                            + (0 if grid[row][col - 1] == 1 else 1),
                        )

            # Backward pass: check cells coming from right and bottom
            for row in range(num_rows - 1, -1, -1):
                for col in range(num_cols - 1, -1, -1):
                    # Check cell below
                    if row < num_rows - 1:
                        min_changes[row][col] = min(
                            min_changes[row][col],
                            min_changes[row + 1][col]
                            + (0 if grid[row + 1][col] == 4 else 1),
                        )
                    # Check cell to the right
                    if col < num_cols - 1:
                        min_changes[row][col] = min(
                            min_changes[row][col],
                            min_changes[row][col + 1]
                            + (0 if grid[row][col + 1] == 2 else 1),
                        )
            # If no changes were made in this iteration, we've found optimal solution
            if min_changes == prev_state:
                break

        return min_changes[num_rows - 1][num_cols - 1]

In [ ]:
from collections import deque
from typing import List

class Solution:
    def minCost(self, grid: List[List[int]]) -> int:
        rows = len(grid)
        cols = len(grid[0])
        n = rows * cols
        goal_idx = n - 1

        # Flatten grid to avoid double indexing
        flat = [grid[r][c] for r in range(rows) for c in range(cols)]

        # Map grid value (1..4) -> preferred direction index (0..3)
        # Direction indices: 0=down, 1=up, 2=right, 3=left
        pref_idx_map = (0, 2, 3, 0, 1)  # index 0 unused

        # Precompute preferred dir per cell (0..3)
        kpref = [pref_idx_map[v] for v in flat]

        # Precompute neighbors per cell (or -1 if invalid)
        down = [-1] * n
        up   = [-1] * n
        right= [-1] * n
        left = [-1] * n

        # Precompute row starts to avoid repeated multiplications
        row_start = [r * cols for r in range(rows)]
        last_row_start = row_start[-1]

        for r in range(rows):
            base = row_start[r]
            # vertical neighbors
            if r > 0:
                for c in range(cols):
                    i = base + c
                    up[i] = i - cols
            if r < rows - 1:
                for c in range(cols):
                    i = base + c
                    down[i] = i + cols
            # horizontal neighbors
            # left edges
            left[base] = -1
            for c in range(1, cols):
                i = base + c
                left[i] = i - 1
            # right edges
            right[base + cols - 1] = -1
            for c in range(cols - 1):
                i = base + c
                right[i] = i + 1

        # Distances (max cost is <= number of moves, so small INF works)
        INF = rows * cols
        dist = [INF] * n
        dist[0] = 0

        dq = deque([0])
        append = dq.append
        appendleft = dq.appendleft
        popleft = dq.popleft

        # Local aliases for speed
        dist_local = dist
        kpref_local = kpref
        down_n = down; up_n = up; right_n = right; left_n = left

        while dq:
            i = popleft()
            cur = dist_local[i]
            if i == goal_idx:
                return cur

            kp = kpref_local[i]

            # k = 0 (down)
            j = down_n[i]
            if j != -1:
                step = 0 if kp == 0 else 1
                nd = cur + step
                if nd < dist_local[j]:
                    dist_local[j] = nd
                    (appendleft if step == 0 else append)(j)

            # k = 1 (up)
            j = up_n[i]
            if j != -1:
                step = 0 if kp == 1 else 1
                nd = cur + step
                if nd < dist_local[j]:
                    dist_local[j] = nd
                    (appendleft if step == 0 else append)(j)

            # k = 2 (right)
            j = right_n[i]
            if j != -1:
                step = 0 if kp == 2 else 1
                nd = cur + step
                if nd < dist_local[j]:
                    dist_local[j] = nd
                    (appendleft if step == 0 else append)(j)

            # k = 3 (left)
            j = left_n[i]
            if j != -1:
                step = 0 if kp == 3 else 1
                nd = cur + step
                if nd < dist_local[j]:
                    dist_local[j] = nd
                    (appendleft if step == 0 else append)(j)

        return -1  # unreachable

"""
from collections import deque
from typing import NamedTuple, Callable, List

class Solution:
    def minCost(self, grid: List[List[int]]) -> int:
        start = Location(0, 0)
        goal = Location(len(grid) - 1, len(grid[0]) - 1)

        direction_map = {
            1: Location(0, 1),
            2: Location(0, -1),
            3: Location(1, 0),
            4: Location(-1, 0),
        }

        def get_cost(from_cell: Location, to_cell: Location) -> int:
            diff = direction_map[grid[from_cell.row][from_cell.col]]
            return 0 if from_cell.row + diff.row == to_cell.row and from_cell.col + diff.col == to_cell.col else 1
                
        return zero_one_bfs_grid(grid, start, goal, get_cost)


class Location(NamedTuple):
    row: int
    col: int

GetCostFn = Callable[[Location, Location], int]

def zero_one_bfs_grid(
    grid: List[List[int]],
    start: Location,
    goal: Location,
    get_cost: GetCostFn
) -> int:
    "
    Finds the minimum cost path from start to goal in a grid using 0–1 BFS.

    grid: 2D list of ints (passed to get_cost for direction/weight logic)
    start, goal: Location(row, col)
    get_cost: function(from, to) -> 0 or 1 (edge cost)
    Returns: minimum cost from start to goal, or -1 if unreachable
    "

    rows, cols = len(grid), len(grid[0])
    INF = 10**18  # A large sentinel value representing infinity

    # distance[r][c] = minimum cost to reach cell (r, c) from start
    distance = [[INF] * cols for _ in range(rows)]
    distance[start.row][start.col] = 0  # cost to reach start is 0

    # Double-ended queue for 0–1 BFS
    queue = deque([start])

    # Possible neighbor moves (up, down, left, right)
    directions = [Location(1,0), Location(-1,0), Location(0,1), Location(0,-1)]

    while queue:
        current = queue.popleft()  # Pop from front (like BFS)
        current_dist = distance[current.row][current.col]

        # If we've reached the goal, return the min cost immediately
        if current == goal:
            return current_dist

        # Explore all neighbors (up to 4)
        for d in directions:
            neighbor = Location(current.row + d.row, current.col + d.col)

            # Skip neighbors outside grid bounds
            if not (0 <= neighbor.row < rows and 0 <= neighbor.col < cols):
                continue

            # Ask the cost function for the step cost (0 or 1)
            step_cost = get_cost(current, neighbor)
            new_dist = current_dist + step_cost

            # If we found a cheaper way to reach the neighbor, update
            if new_dist >= distance[neighbor.row][neighbor.col]:
                continue

            distance[neighbor.row][neighbor.col] = new_dist

            # Optimization of 0–1 BFS:
            #   - If edge cost = 0, push to the *front* of deque
            #     so it gets processed sooner.
            #   - If edge cost = 1, push to the *back* of deque.
            if step_cost == 0:
                queue.appendleft(neighbor)
            else:
                queue.append(neighbor)

    # If we emptied the queue without reaching goal → unreachable
    return -1
"""